In [2]:
%pip install fastembed

  Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 6.0 MB/s eta 0:00:00a 0:00:01
Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl (2.7 MB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 7.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 7.2 MB/s eta 0:00:0000:0100:01
Using cached PyYAML-6.0.2-cp311-cp311-macosx_11_0_arm64.whl (172 kB)
Using cached filelock-3.18.0-py3-non

In [3]:
from fastembed import TextEmbedding

/Users/marinazdanova/DataTalks_LLM2025/DataCamp2025_LLM/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
## Q1. Embedding the query

# Define the query
query = "I just discovered the course. Can I join now?"

# Create the embedding model
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Get the embedding (as a generator, so convert to list/array)
import numpy as np
embedding = next(model.embed([query]))
embedding = np.array(embedding)

# Check the shape
print(f"Embedding shape: {embedding.shape}")

# Find the minimal value
min_value = embedding.min()
print(f"Minimal value in the embedding: {min_value}")

min_value

Embedding shape: (512,)
Minimal value in the embedding: -0.11726373551188797


np.float64(-0.11726373551188797)

In [6]:
## Q2. Cosine similarity with another vector

# Embed the document

doc = 'Can I still join the course after the start date?'
doc_embedding = next(model.embed([doc]))
doc_embedding = np.array(doc_embedding)

# Compute cosine similarity between query and document embeddings
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

similarity = cosine_similarity(embedding, doc_embedding)
print(f"Cosine similarity between query and document: {similarity}")
similarity

Cosine similarity between query and document: 0.9008528856818038


np.float64(0.9008528856818038)

In [10]:
## Q3. Ranking by cosine

# List of documents

documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

# Compute embeddings for all document texts
doc_texts = [doc['text'] for doc in documents]
doc_embeddings = list(model.embed(doc_texts))
doc_embeddings = np.array(doc_embeddings)
print(f"Document embeddings shape: {doc_embeddings.shape}")

# Compute cosine similarities with the query embedding
similarities = [cosine_similarity(embedding, doc_emb) for doc_emb in doc_embeddings]
print(f"Cosine similarities: {similarities}")

# Find the index with the highest similarity
best_idx = int(np.argmax(similarities))
print(f"Document index with highest similarity: {best_idx}")
best_idx

Document embeddings shape: (5, 512)
Cosine similarities: [np.float64(0.7629684493123693), np.float64(0.8182378361919107), np.float64(0.8085397290762828), np.float64(0.7133078539597724), np.float64(0.7304499528359614)]
Document index with highest similarity: 1


1

In [11]:
## Q4. Ranking by cosine, version two

# Compute embeddings for concatenated question + text fields
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
full_embeddings = list(model.embed(full_texts))
full_embeddings = np.array(full_embeddings)
print(f"Full text embeddings shape: {full_embeddings.shape}")

# Compute cosine similarities with the query embedding
full_similarities = [cosine_similarity(embedding, full_emb) for full_emb in full_embeddings]
print(f"Cosine similarities (full_text): {full_similarities}")

# Find the index with the highest similarity
best_full_idx = int(np.argmax(full_similarities))
print(f"Document index with highest similarity (full_text): {best_full_idx}")
best_full_idx

Full text embeddings shape: (5, 512)
Cosine similarities (full_text): [np.float64(0.8514542983476503), np.float64(0.8436594058703555), np.float64(0.8408287206920104), np.float64(0.7755157689596764), np.float64(0.8086007942697778)]
Document index with highest similarity (full_text): 0


0

In [12]:
# Install required libraries
%pip install fastembed qdrant-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/10.6 MB ? eta -:--:--Downloading grpcio-1.73.0-cp311-cp311-macosx_11_0_universal2.whl (10.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 7.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.9 MB ? eta -:--:--Downloading pydantic_core-2.33.2-cp311-cp311-macosx_11_0_arm64.whl (1.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [qdrant-client]0m [qdrant-client]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [qdrant-client]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:
## Q6. Indexing with qdrant (2 points)

# Download the FAQ records for machine-learning-zoomcamp
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []
for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

print(f"Number of ML Zoomcamp FAQ records: {len(documents)}")

Number of ML Zoomcamp FAQ records: 375


In [14]:
# Prepare the embedding model
from fastembed import TextEmbedding

model = TextEmbedding(model_name="BAAI/bge-small-en")

Fetching 5 files: 100%|██████████| 5/5 [00:18<00:00,  3.80s/it]



In [15]:
# Prepare Qdrant client and collection
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

qdrant = QdrantClient(":memory:")  # Use in-memory Qdrant for demo; replace with host/port for persistent

collection_name = "ml_zoomcamp_faq"
qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

/var/folders/vd/dl48s8js5_z0k6_rb5jv19xw0000gn/T/ipykernel_19517/1169140143.py:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [16]:
# Add documents to Qdrant
from tqdm import tqdm

payloads = []
vectors = []
for doc in tqdm(documents):
    text = (doc.get("question", "") or "") + " " + (doc.get("text", "") or "")
    payloads.append(doc)
    # Get embedding for the concatenated text
    vector = next(model.embed([text]))
    vectors.append(vector)

qdrant.upload_collection(
    collection_name=collection_name,
    vectors=vectors,
    payload=payloads,
    ids=None,  # Let Qdrant auto-assign IDs
    batch_size=64
)
print("Documents uploaded to Qdrant.")

100%|██████████| 375/375 [00:06<00:00, 54.78it/s]

Documents uploaded to Qdrant.


In [23]:
# Query the collection with the question from Q1
query = "I just discovered the course. Can I join now?"
query_vector = next(model.embed([query]))

search_result = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=5
)

print(f"Number of results found: {len(search_result)}")

print(f"Highest score in the results: {search_result[0].score:.2f}")
search_result[0].score

# print first documents and scores
for result in search_result:
    print(f"Document ID: {result.id}\nScore: {result.score:.2f}\nPayload: {result.payload}\n\n")

Number of results found: 5
Highest score in the results: 0.87
Document ID: d4ec3b0e-3739-440c-a7f0-3abcfd548ca8
Score: 0.87
Payload: {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}


Document ID: 646c9c4e-4ad8-47a8-bba1-edeb33eacef4
Score: 0.87
Payload: {'text': 'Approximately 4 months, but may take more if you want to do some extra activities (an extra project, an article, etc)', 'section': 'General course-related questions', 'question': 'How long is the course?', 'course': 'machine-learning-zoomcamp'}

/var/folders/vd/dl48s8js5_z0k6_rb5jv19xw0000gn/T/ipykernel_19517/2061679989.py:5: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant.search(


The highest score in the results will be printed above. Based on typical results for this setup, the closest answer is **0.87**.